<a href="https://colab.research.google.com/github/KatsuFujii/Scraping_practice/blob/main/keiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import re
class CollectRaceIDList:
    @staticmethod
    def collectRaceID(collect_id):
      this_year = 2022
      search_range = 10
      search_start = this_year - search_range 
      collect_id_list = [] 
      for year in range(search_start,this_year):
        id = str(year) + collect_id
        collect_id_list.append(id)
      return collect_id_list

class CollectHorseData:
    @staticmethod
    def collectHorse(race_id):
      url = "https://race.netkeiba.com/race/shutuba.html?race_id=" + race_id
      #メインとなるテーブルデータを取得
      df = pd.read_html(url)[0]

      html = requests.get(url)
      html.encoding = "EUC-JP"
      soup = BeautifulSoup(html.text, "html.parser")
      horse_id_list = []
      horse_a_list = soup.find("table", attrs={"class": "Shutuba_Table"}).find_all("a", attrs={"href": re.compile("https://db.netkeiba.com/horse")})
      for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                    
      return horse_id_list
class Results:
    @staticmethod
    def scrape(race_id_list):
        """
        レース結果データをスクレイピングする関数

        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト

        Returns:
        ----------
        race_results_df : pandas.DataFrame
            全レース結果データをまとめてDataFrame型にしたもの
        """

        #race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
          time.sleep(1)
          try:
                url = "https://db.netkeiba.com/race/" + race_id
                #メインとなるテーブルデータを取得
                df = pd.read_html(url)[0]

                html = requests.get(url)
                html.encoding = "EUC-JP"
                soup = BeautifulSoup(html.text, "html.parser")

                #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                texts = (
                    soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                for text in info:
                    if text in ["芝", "ダート"]:
                        df["race_type"] = [text] * len(df)
                    if "障" in text:
                        df["race_type"] = ["障害"] * len(df)
                    if "m" in text:
                        df["course_len"] = [int(re.findall(r"\d+", text)[-1])] * len(df)
                    if text in ["良", "稍重", "重", "不良"]:
                        df["ground_state"] = [text] * len(df)
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df["weather"] = [text] * len(df)
                    if "年" in text:
                        df["date"] = [text] * len(df)

                #馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])
                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list
                rap = soup.find("table", attrs={"summary": "ラップタイム"}).find_all("td",class_="race_lap_cell")[0].text
                df["ラップ"] = rap
                #インデックスをrace_idにする
                df.index = [race_id] * len(df)

                race_results[race_id] = df
            #存在しないrace_idを飛ばす
          except IndexError:
                continue
          except AttributeError: #存在しないrace_idでAttributeErrorになるページもあるので追加
                continue
            #wifiの接続が切れた時などでも途中までのデータを返せるようにする
          except Exception as e:
                print(e)
                break
            #Jupyterで停止ボタンを押した時の対処
          except:
                break

        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])

        return race_results_df

        #馬の過去成績データを処理するクラス
class HorseResults:
    @staticmethod
    def scrape(horse_id_list):
        """
        馬の過去成績データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        horse_results_df : pandas.DataFrame
            全馬の過去成績データをまとめてDataFrame型にしたもの
        """

        #horse_idをkeyにしてDataFrame型を格納
        horse_results = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = 'https://db.netkeiba.com/horse/' + horse_id

                df = pd.read_html(url)[3]
                #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0]=='受賞歴':
                    df = pd.read_html(url)[4]
                html = requests.get(url)
                html.encoding = "EUC-JP"
                soup = BeautifulSoup(html.text, "html.parser")
                horse_name = soup.find("div",attrs={"class": "horse_title"}).find("h1").text
                print(horse_name)
                rap_list = []
                race_list = soup.find("table", attrs={"class": "db_h_race_results"}).find_all("a", attrs={"href": re.compile("^/race/\d")})
                for a in race_list:
                  try:
                    race_url = "https://db.netkeiba.com" + a.get("href")
                    race_html = requests.get(race_url)
                    race_html.encoding = "EUC-JP"
                    race_soup = BeautifulSoup(race_html.text, "html.parser")
                    elm = race_soup.find("table", attrs={"summary": "ラップタイム"}).find_all("td",class_="race_lap_cell")
                    if(elm != []):
                      rap = elm[0].text
                    else:
                      rap = "データなし"
                    rap_list.append(rap)
                    time.sleep(1)
                  except IndexError:
                      continue
                  except:
                    break
                df["ラップ"] = rap_list
                df.index = [horse_name] * len(df)
                horse_results[horse_id] = df
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #pd.DataFrame型にして一つのデータにまとめる        
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])

        return horse_results_df

01:札幌　02：函館　03：福島　04：新潟　05：東京　06：中山　07：中京　08：京都　09：阪神　10：小倉

In [ ]:
#レース分析用
#collect_id = "05030211"
#Results.scrape(CollectRaceIDList.collectRaceID(collect_id)).to_csv("./race_analysis.csv",line_terminator='\r\n',encoding='utf-8-sig')

#ラップ分析用
race_id="202205030211"
#結果をcsv出力
HorseResults.scrape(CollectHorseData.collectHorse(race_id)).to_csv("./keiba_results.csv",line_terminator='\r\n',encoding='utf-8-sig')